In [25]:
# ruff: noqa: E501
import unittest
from dataclasses import astuple, dataclass

import numpy as np
import pyvista
import tetgen
from parameterized import parameterized
from pyvista import PolyData
from scipy.optimize import minimize

from irregular_object_packing.mesh.transform import scale_and_center_mesh
from irregular_object_packing.packing.chordal_axis_transform import (
    compute_cat_faces,
    face_coord_to_points_and_faces,
)
from irregular_object_packing.packing.nlc_optimisation import (
    compute_optimal_growth,
    construct_transform_matrix,
    construct_transform_matrix_from_array,
    local_constraint_multiple_points,
    objective,
    transform_v,
)

RB = 1 / 12 * np.pi


class TestObj:
    def __init__(self) -> None:
        self.obj_points = np.array(
            [
                [2, 1, 1],
                [11, 3, 1],
                [10, 7, 1],
                [1, 5, 1],
                [2, 1, 3],
                [11, 3, 3],
                [10, 7, 3],
                [1, 5, 3],
            ], dtype=np.float64)
        self.init_center = np.mean(self.obj_points, axis=0)
        self.obj_faces = np.hstack(np.array([
            [4, 0, 1, 2, 3],
            [4, 4, 5, 6, 7],
            [4, 0, 1, 5, 4],
            [4, 1, 2, 6, 5],
            [4, 2, 3, 7, 6],
            [4, 3, 0, 4, 7],
        ]), )

        self.container_points = np.array([
            [0, 0, 0],
            [15, 0, 0],
            [15, 10, 0],
            [0, 10, 0],
            [0, 0, 4],
            [15, 0, 4],
            [15, 10, 4],
            [0, 10, 4],
        ], dtype=np.float64)
        self.container_center = np.mean(self.container_points, axis=0)
        self.container_faces = self.obj_faces.copy()
        self.obj = PolyData(self.obj_points, self.obj_faces)
        self.container = PolyData(self.container_points, self.container_faces)
        self.obj0 = scale_and_center_mesh(self.obj, self.obj.volume)

        self.tet_input = (self.container + self.obj).triangulate()

        tet = tetgen.TetGen(self.tet_input)
        tet.tetrahedralize(order=1, mindihedral=0, minratio=0, steinerleft=0, quality=False)

        self.cat_data = compute_cat_faces(
            tet.grid, [set(map(tuple, self.obj_points)), set(map(tuple, self.container_points))],
            [self.init_center],
        )

        self.obj_cat_cell = PolyData(*face_coord_to_points_and_faces(self.cat_data, 0))
        self.previous_transform_array = np.array([1, 0, 0, 0] + list(self.init_center))

    def test_optimize_cat_box(self):
        self.new_tf_array = compute_optimal_growth(
            0,
            self.previous_transform_array,
            5,
            (0, None),
            1 / 12 * np.pi,
            None,
            0,
            cat_data=self.cat_data,
        )

        # transform the object
        new_obj = self.obj0.transform(construct_transform_matrix_from_array(self.new_tf_array), inplace=False)

        inside = new_obj.select_enclosed_points(self.container)
        pts = new_obj.extract_points(inside['SelectedPoints'].view(bool), adjacent_cells=False)

        plotter = pyvista.Plotter()
        plotter.add_mesh(self.obj, color='green', opacity=0.9)
        plotter.add_mesh(self.obj_cat_cell, color='yellow', opacity=0.8)
        plotter.add_mesh(inside, color='red', opacity=0.8)
        plotter.add_mesh(self.container, color='blue', opacity=0.5)
        plotter.show()


test_obj = TestObj()
test_obj.test_optimize_cat_box()


Widget(value="<iframe src='http://localhost:59448/index.html?ui=P_0x17d0fe380_12&reconnect=auto' style='width:…

In [20]:
test_obj.init_center


array([6., 4., 2.])

In [17]:
test_obj.new_tf_array


array([ 2.37302416e+00, -2.55828127e-12, -5.28262607e-13, -2.07987064e-01,
        6.75000000e+00,  4.50000000e+00,  2.00000000e+00])